In [1]:
import sys,os
path = '/nfs/team283/aa16/KR_NAS/'
os.chdir(path)
sys.path.append(path)

In [2]:
%pylab inline
import pandas as pd
import NaiveDE
import SpatialDE
import statsmodels.stats.multitest as multi
import scipy.stats as ss
import pickle as pickle
import scipy as sc
import matplotlib
import statsmodels.stats.multitest as multi
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

Populating the interactive namespace from numpy and matplotlib


Load data (without negative probes):

In [3]:
adata = pickle.load(open('data/nanostringWTA_fetailBrain_AnnData.p',  "rb"))
adata = adata[:,adata.var_names != 'NegProbe-WTX']
adata.layers['Stabilized'] = NaiveDE.stabilize(adata.X)
adata.obs['TotalCounts'] = np.sum(adata.X, axis = 1)
adata.layers['TotalCountsRegressed'] = NaiveDE.regress_out(adata.obs, adata.layers['Stabilized'].T, 'np.log(TotalCounts)').T

/nfs/users/nfs_a/aa16/.local/lib/python3.7/site-packages/anndata/_core/anndata.py:21: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex


In [4]:
adata.layers

Layers with keys: CPM, X_Corrected, CPM_corrected, X, Stabilized, TotalCountsRegressed

Order AOIs according to slide, radial_position and cortical depth, returning a data frame with the index of each HOPXpos, EOMESpos and background AOI for each position, as well as the parallel geometric ROI.

In [11]:
celltypeAOIs_1 = np.array(np.where([adata.obs['Radial_position'][i] in (1,3) and adata.obs['AOI_type'][i] == 'EOMESpos'
                for i in range(len(adata.obs['age']))])[0])

celltypeAOIs_2 = np.array(np.where([adata.obs['Radial_position'][i] in (1,3) and adata.obs['AOI_type'][i] == 'HOPXpos'
                for i in range(len(adata.obs['age']))])[0])

backgroundAOIs = np.array(np.where([adata.obs['Radial_position'][i] in (1,3) and adata.obs['AOI_type'][i] == 'Residual'
                for i in range(len(adata.obs['age']))])[0])

parallelROIs = np.array(np.where([adata.obs['Radial_position'][i] == 2 and adata.obs['AOI_type'][i] == 'Geometric'
                for i in range(len(adata.obs['age']))])[0])

ringROIs = np.array(np.where([adata.obs['Radial_position'][i] in (1,3) and adata.obs['AOI_type'][i] == 'Ring'
                for i in range(len(adata.obs['age']))])[0])

In [15]:
celltypeAOIs_1 = np.array(np.where([adata.obs['slide'][i] in ('00MU', '00MV', '00MV-2', '00MW') 
                and adata.obs['Radial_position'][i] in (1,3) and adata.obs['AOI_type'][i] == 'EOMESpos'
                for i in range(len(adata.obs['age']))])[0])

celltypeAOIs_2 = np.array(np.where([adata.obs['slide'][i] in ('00MU', '00MV', '00MV-2', '00MW') 
                and adata.obs['Radial_position'][i] in (1,3) and adata.obs['AOI_type'][i] == 'HOPXpos'
                for i in range(len(adata.obs['age']))])[0])

backgroundAOIs = np.array(np.where([adata.obs['slide'][i] in ('00MU', '00MV', '00MV-2', '00MW') 
                and adata.obs['Radial_position'][i] in (1,3) and adata.obs['AOI_type'][i] == 'Residual'
                for i in range(len(adata.obs['age']))])[0])

parallelROIs = np.array(np.where([adata.obs['slide'][i] in ('00MU', '00MV', '00MV-2', '00MW') 
                and adata.obs['Radial_position'][i] == 2 and adata.obs['AOI_type'][i] == 'Geometric'
                for i in range(len(adata.obs['age']))])[0])

ringAOIs = np.array(np.where([adata.obs['slide'][i] in ('00MU', '00MV', '00MV-2', '00MW') 
                and adata.obs['Radial_position'][i] in (1,3) and adata.obs['AOI_type'][i] == 'Ring'
                for i in range(len(adata.obs['age']))])[0])

adata.obs.loc[adata.obs['slide'] == '00MV-2', 'slide'] = '00MV'

order_list1 = []
order_list2 = []
order_list3 = []
order_list4 = []

unique_slides = np.array(('00MU', '00MV', '00MW'))
unique_positions = (1,3)
combinations = []

for i in range(len(unique_slides)):
    for j in range(len(unique_positions)):
        order_list1.append(backgroundAOIs[[adata.obs['slide'][k] in unique_slides[i] and 
                                    adata.obs['Radial_position'][k] == unique_positions[j]
                           for k in backgroundAOIs]])
        combinations.append((unique_slides[i], unique_positions[j]))

for i in range(len(unique_slides)):
    for j in range(len(unique_positions)):
        order_list2.append(celltypeAOIs_1[[adata.obs['slide'][k] in unique_slides[i] and 
                                    adata.obs['Radial_position'][k] == unique_positions[j]
                           for k in celltypeAOIs_1]])

for i in range(len(unique_slides)):
    for j in range(len(unique_positions)):
        order_list3.append(celltypeAOIs_2[[adata.obs['slide'][k] in unique_slides[i] and 
                                    adata.obs['Radial_position'][k] == unique_positions[j]
                           for k in celltypeAOIs_2]])
        
for i in range(len(unique_slides)):
    for j in range(len(unique_positions)):
        order_list4.append(ringAOIs[[adata.obs['slide'][k] in unique_slides[i] and 
                                    adata.obs['Radial_position'][k] == unique_positions[j]
                           for k in ringAOIs]])

VCDepths = list()
for i in range(len(order_list1)):
    VCDepths.append(np.intersect1d(np.intersect1d(np.intersect1d(adata.obs['VCDepth'].iloc[order_list1[i]],
                                                  adata.obs['VCDepth'].iloc[order_list2[i]]),
                   adata.obs['VCDepth'].iloc[order_list3[i]]),adata.obs['VCDepth'].iloc[order_list4[i]]))

input_data = pd.DataFrame(columns = ('Slide', 'Radial_position', 'VCDepth', 'Background', 'HOPXpos',
                                     'EOMESpos', 'ParallelROI', 'Ring'),
             index = range(sum([len(VCDepths[i]) for i in range(len(VCDepths))])))

input_data['VCDepth'] = [item for sublist in VCDepths for item in sublist]

slides = []
radial_positions = []
for i in range(len(VCDepths)):
    slides.append(np.repeat(combinations[i][0], len(VCDepths[i])))
    radial_positions.append(np.repeat(combinations[i][1], len(VCDepths[i])))

input_data['Slide'] = [item for sublist in slides for item in sublist]
input_data['Radial_position'] = [item for sublist in radial_positions for item in sublist]

# Finally fill in the relevant index in the adata frame:
for i in input_data.index:
    
    input_data['Background'].iloc[i] = np.where([adata.obs['slide'][j] == input_data['Slide'][i] and
                                                adata.obs['Radial_position'][j] == input_data['Radial_position'][i] and
                                                adata.obs['VCDepth'][j] == input_data['VCDepth'][i] and
                                                adata.obs['AOI_type'][j] == 'Residual' 
                                                for j in range(len(adata.obs['slide']))])[0][0]
    
    input_data['HOPXpos'].iloc[i] = np.where([adata.obs['slide'][j] == input_data['Slide'][i] and
                                                adata.obs['Radial_position'][j] == input_data['Radial_position'][i] and
                                                adata.obs['VCDepth'][j] == input_data['VCDepth'][i] and
                                                adata.obs['AOI_type'][j] == 'HOPXpos' 
                                                for j in range(len(adata.obs['slide']))])[0][0]

    input_data['EOMESpos'].iloc[i] = np.where([adata.obs['slide'][j] == input_data['Slide'][i] and
                                                adata.obs['Radial_position'][j] == input_data['Radial_position'][i] and
                                                adata.obs['VCDepth'][j] == input_data['VCDepth'][i] and
                                                adata.obs['AOI_type'][j] == 'EOMESpos' 
                                                for j in range(len(adata.obs['slide']))])[0][0]    
    
    input_data['Ring'].iloc[i] = np.where([adata.obs['slide'][j] == input_data['Slide'][i] and
                                                adata.obs['Radial_position'][j] == input_data['Radial_position'][i] and
                                                adata.obs['VCDepth'][j] == input_data['VCDepth'][i] and
                                                adata.obs['AOI_type'][j] == 'Ring' 
                                                for j in range(len(adata.obs['slide']))])[0][0]    

for i in range(np.shape(input_data)[0]):
    index = np.where(np.array(adata.obs['slide']) == input_data.iloc[i,0])[0]
    index = [i for i in index if i in parallelROIs]
    input_data.iloc[i,6] = index[np.argmin(np.abs(np.array(adata.obs['VCDepth'].iloc[index]) - input_data['VCDepth'].iloc[i]))]     
    
print(input_data)

pickle.dump(input_data, open( "data/ROI_relations.p", "wb" ))

   Slide  Radial_position   VCDepth Background HOPXpos EOMESpos ParallelROI  \
0   00MU                1  0.028892        126     111      103         107   
1   00MU                1  0.094703        156     141      134         122   
2   00MU                1  0.205457         96     172      164         137   
3   00MU                1  0.303371        127     112      104         152   
4   00MU                1  0.382022        157     142      135         160   
5   00MU                3  0.028892         94     170      162         107   
6   00MU                3  0.094703        125     110      102         122   
7   00MU                3  0.205457        155     140      133         137   
8   00MU                3  0.303371         95     171      163         152   
9   00MU                3  0.382022         97     173      165         160   
10  00MV                1  0.312557         35      21       13          98   
11  00MV                1  0.392513         67      